<a href="https://colab.research.google.com/github/armandossrecife/my_validation3/blob/main/my_analysis_activemq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup do Ambiente de Análise

In [ ]:
!rm -rf *.log
!rm -rf *.txt
!rm -rf *.xlsx
!rm -rf my_issues
!rm -rf activemq

In [ ]:
print('Install Pydriller.')
!pip install pydriller > install_pydriller.log
print('Install gitpython.')
!pip3 install gitpython > install_gitpython.log
print('Install Jira Python lib.')
!pip install jira > install_jira_python.log
print('Install SQLite in Linux')
!sudo apt install -y sqlite3 > install_sqlite.log
print('All depenpencies installed!')
!cat *.log > install.log
print('Details in install.log')

Install Pydriller.
Install gitpython.
Install Jira Python lib.
Install SQLite in Linux


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
All depenpencies installed!
Details in install.log


## Importa bibliotecas necessárias

In [ ]:
import os
from datetime import datetime, timedelta
import tqdm
from pydriller import Repository
from jira import JIRA
import pandas as pd
import re
import scipy.stats as stats
import random
import shutil
import time

## Variáveis Globais

In [ ]:
my_repository = 'activemq'
url_to_repository = 'https://github.com/apache/activemq.git'
os.environ['MY_REPOSITORY'] = url_to_repository
os.environ['MY_PATH_REPOSITORY'] = my_repository

JIRA_SERVER = 'https://issues.apache.org/jira'
ISSUE_TRACKER_PROJECT = 'AMQ'

# Credentials
os.environ['USERNAME'] = '?'
os.environ['PASSWORD'] = '?'
username = os.environ.get('USERNAME')
password = os.environ.get('PASSWORD')

arquivos_activemq_by_sysrepoanalysis = ['BrokerService.java',
 'DemandForwardingBridgeSupport.java',
 'MessageDatabase.java',
 'Queue.java',
 'TopicSubscription.java',
 'TransportConnector.java']

lista_arquivos_criticos = ['DemandForwardingBridgeSupport.java',
'SubQueueSelectorCacheBroker.java',
'BrokerServiceAware.java',
'TransportConnector.java',
'BrokerService.java',
'TopicSubscription.java',
'QueueBrowserSubscription.java',
'Queue.java',
'DurableTopicSubscription.java',
'QueueSubscription.java',
'QueueDispatchSelector.java',
'PendingQueueMessageStoragePolicy.java',
'MirroredQueue.java',
'MessageQueue.java']


## Clona o Repositório

In [ ]:
print(f'Clona o repositório : {url_to_repository}')
!git clone $MY_REPOSITORY

Clona o repositório : https://github.com/apache/activemq.git
Cloning into 'activemq'...
remote: Enumerating objects: 237396, done.
remote: Counting objects: 100% (4162/4162), done.
remote: Compressing objects: 100% (1954/1954), done.
remote: Total 237396 (delta 1763), reused 3856 (delta 1586), pack-reused 233234
Receiving objects: 100% (237396/237396), 61.57 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (114761/114761), done.


In [ ]:
!cd $MY_PATH_REPOSITORY && git log --pretty="%H %s" > all_commits_msg.txt
!cd $MY_PATH_REPOSITORY && git log --pretty="%H;%ai;%s" > all_commits_full.txt

# Dados do 1o commit e do ultimo commit
!echo "Último commit: "
!head /content/$MY_PATH_REPOSITORY/all_commits_full.txt -n 1
!echo "Primeiro commit: "
!tail /content/$MY_PATH_REPOSITORY/all_commits_full.txt -n 1

Último commit: 
5dbe5f1c410b4fef0e93a2534f3eeaa8f2349ae2;2023-11-01 14:51:01 +0100;Fix OSGi headers
Primeiro commit: 
40a7d3b6ac35d2ecb34e85fc3403d2e48e33874e;2005-12-12 17:53:59 +0000;Moved the trunk code into the trunk sub directory


## Funções de apoio

In [ ]:
def get_all_commits_by_range(initial_date, final_date, repository_name):
  """Extracts information from a date range of commits
  Args:
      initial_date: inicial date of commits
      final_date: final date of commits
  Returns:
        A dictionary of commits, where the keys are the commit hashes and the
        values are tuples containing the commit message, commit's date, commit's line, commit's file, a list of files modified in the commit, a list of all diffs from modified files
  """
  dict_commit_modified_files = {}
  print('Wait...')
  my_traverser_commits = Repository(repository_name, since=initial_date, to=final_date).traverse_commits()
  total_commits = len(list(my_traverser_commits))
  try:
    for commit in tqdm.tqdm(Repository(repository_name, since=initial_date, to=final_date).traverse_commits(), total=total_commits, desc="Progress commit anlysis"):
      list_of_modified_files = []
      list_dict_of_diff_modified_files = []
      for m in commit.modified_files:
        dict_m_diff = {}
        if m is not None:
          list_of_modified_files.append(m.filename)
          dict_m_diff[m.filename] = m.diff
          list_dict_of_diff_modified_files.append(dict_m_diff)
      data_commit = str(commit.committer_date.day) + '/' + str(commit.committer_date.month) + '/' + str(commit.committer_date.year)
      element = commit.msg, data_commit, commit.lines, commit.files, list_of_modified_files, list_dict_of_diff_modified_files
      dict_commit_modified_files[commit.hash] = element
  except Exception as ex:
    print(f'Erro during travesse commits: {str(ex)}')
  return dict_commit_modified_files, total_commits

def get_commits_by_range_and_critical_files(initial_date, final_date, critical_files, repository_name):
  """Extracts information from a date range of commits, focusing on critical files.
  Args:
      initial_date: inicial date of commits
      final_date: final date of commits
      critical_files: A list of critical file paths. ex: ['StorageService.java', 'ColumnFamilyStore.java']
  Returns:
        A dictionary of commits, where the keys are the commit hashes and the
        values are tuples containing the commit message, commits's date, commit's line, commit's file, a list of critical files modified in the commit, a list_of_modified_files, a dictionary of diff [filename]:filename.diff, a list of all diffs from modified files
        commit.msg, data_commit, commit.lines, commit.files, list_of_critical_files_modified, list_of_modified_files, list_dict_of_diff_files, list_dict_of_diff_modified_files
  """

  dict_commit_modified_files = {}
  print('Wait...')
  my_traverser_commits = Repository(repository_name, since=initial_date, to=final_date).traverse_commits()
  total_commits = len(list(my_traverser_commits))
  try:
    for commit in tqdm.tqdm(Repository(repository_name, since=initial_date, to=final_date).traverse_commits(), total=total_commits, desc="Progress critical files commit anlysis"):
      list_of_critical_files_modified = []
      list_of_modified_files = []
      list_dict_of_diff_files = []
      list_dict_of_diff_modified_files = []
      for m in commit.modified_files:
        dict_diff_file = {}
        list_of_modified_files.append(m.filename)
        list_dict_of_diff_modified_files.append(m.diff)
        if m.filename in critical_files:
          list_of_critical_files_modified.append(m.filename)
          dict_diff_file[m.filename] = m.diff
          list_dict_of_diff_files.append(dict_diff_file)
      if len(list_of_critical_files_modified) > 0:
        data_commit = str(commit.committer_date.day) + '/' + str(commit.committer_date.month) + '/' + str(commit.committer_date.year)
        element = commit.msg, data_commit, commit.lines, commit.files, list_of_critical_files_modified, list_of_modified_files, list_dict_of_diff_files, list_dict_of_diff_modified_files
        dict_commit_modified_files[commit.hash] = element
  except Exception as ex:
    print(f'Erro during travesse commits: {str(ex)}')
  return dict_commit_modified_files, total_commits

In [ ]:
class JiraIssue:
  def __init__(self, key, summary, issue_type, status, priority, description, comments, created_date=None, updated_date=None, resolved_date=None):
    self.key = key
    self.summary = summary
    self.issue_type = issue_type
    self.status = status
    self.priority = priority
    self.description = description
    self.comments = comments
    self.created_date = created_date
    self.updated_date = updated_date
    self.resolved_date = resolved_date

  def get_comments(self):
    return self.comments

  def __str__(self):
    return (f'Key: {self.key}, Summary: {self.summary}, Type: {self.issue_type}, Status: {self.status}')

class JiraIssues:
  def __init__(self,project, issues):
    self.project = project
    self.issues = issues

  def add_issue(self, issue):
    self.issues.append(issue)

  def get_issues(self) -> list:
    return self.issues

  def update_issues(self, issues):
    self.issues = issues

  def __str__(self):
    str_issues = ""
    for issue in self.get_issues():
      str_issues = str_issues + str(issue)
      str_issues = str_issues + ', '
    str_issues = '[' + str_issues + ']'
    return (f'Project: {self.project}, Qdt of issues: {len(self.issues)}, Issues: {str_issues}')

# Classe de utilidades para manipular o servidor Jira
class JiraUtils:
  def __init__(self, project, jira_instance):
    self.project = project
    self.jira_jira_instance = jira_instance

  def generate_intervals_between_dates(self, date1: tuple, date2: tuple, distance=120) -> list:
    start_date = datetime(date1[0], date1[1], date1[2])
    end_date = datetime(date2[0], date2[1], date2[2])
    interval_days = distance
    # Initialize a list to store the intervals
    intervals = []
    # Initialize the current date as the start date
    current_date = start_date
    # Loop to generate intervals until the current date is less than or equal to the end date
    while current_date < end_date:
        interval = (current_date, current_date + timedelta(days=interval_days - 1))
        intervals.append(interval)
        current_date += timedelta(days=interval_days)
    return intervals

  def convert_interval_dates(self, dates: list) -> list:
    list_interval_dates = []
    for each in dates:
      date1 = each[0]
      # Convert the date to a string in the format "YYYY/MM/DD".
      str_date1 = date1.strftime("%Y/%m/%d")
      date2 = each[1]
      str_date2 = date2.strftime("%Y/%m/%d")
      elemento = str_date1, str_date2
      list_interval_dates.append(elemento)
    return list_interval_dates

  def generate_list_of_sentences(self, dates: list) -> list:
    lista_sentencas = []
    for each in dates:
      str_date1 = each[0].strftime("%Y/%m/%d")
      str_date2 = each[1].strftime("%Y/%m/%d")
      sentenca = f'project={self.project.upper()} and created>="{str_date1}" and created<="{str_date2}"'
      lista_sentencas.append(sentenca)
    return lista_sentencas

  def get_list_of_block_issues_by_dates(self,date1, date2, distance=120) -> list:
    print('Aguarde...')
    t1 = datetime.now()
    list_of_dates = self.generate_intervals_between_dates(date1,date2,distance)
    lista_sentencas = self.generate_list_of_sentences(list_of_dates)
    lista_bloco_issues_by_date = []
    total_items = len(lista_sentencas)
    i = 0
    iterable_lista_sentencas = tqdm.tqdm(lista_sentencas, total=total_items)
    for each in iterable_lista_sentencas:
      issues_by_date_temp = self.jira_jira_instance.search_issues(each,maxResults=1000)
      print(f'Range: {each}, qtd issues: {len(issues_by_date_temp)}')
      lista_bloco_issues_by_date.append(issues_by_date_temp)
      percentage = (i + 1) / total_items * 100
      iterable_lista_sentencas.set_description(f"Progress Message Analysis")
    i += 1
    t2 = datetime.now()
    print(t2)
    print(f'Tempo da consulta: {t2-t1}')
    return lista_bloco_issues_by_date

  def concatenate_block_of_issues(self,block_of_issues):
    concatenated_list = [item for sublist in block_of_issues for item in sublist]
    print(f'Total de issues recuperados: {len(concatenated_list)}')
    return concatenated_list

def analyze_jira_all_issues(project, all_issues):
    """Extracts all issues from Issue Tracker
        Args:
          project: Issue Tracker Project name, example: CASSANDRA
          all_issues: a block of issues
        Returns:
          A list of JiraIssues
    """
    # Create an instance of JiraIssues to manage all issues
    my_all_issues = JiraIssues(project, [])
    total_items = len(all_issues)

    # Iterate through the fetched issues
    for issue in tqdm.tqdm(all_issues, total=total_items, desc='Progress jira all issues analysis'):
        issue_key = issue.key
        issue_summary = issue.fields.summary
        issue_description = issue.fields.description
        issue_comments = [comment.body for comment in issue.fields.comment.comments]
        issue_created_date = issue.fields.created
        issue_updated_date = None
        issue_resolved_date = issue.fields.resolutiondate

        # Check for SATD keywords in the issue's summary, description, and comments
        if issue_summary is None:
            issue_summary = ""
        if issue_description is None:
            issue_description = ""
        if issue_comments is None:
            issue_comments = ""
        if issue_created_date == None:
            issue_created_date = ""
        if issue_updated_date == None:
            issue_updated_date = ""
        if issue_resolved_date == None:
            issue_resolved_date = ""

        issue_type = issue.fields.issuetype.name
        issue_status = issue.fields.status.name
        issue_priority = issue.fields.priority.name

        # Create a JiraIssue instance for the SATD issue
        my_issue = JiraIssue(issue_key, issue_summary, issue_type, issue_status, issue_priority, issue_description, issue_comments, issue_created_date, issue_updated_date, issue_resolved_date)

        my_all_issues.add_issue(my_issue)

    return my_all_issues

def convert_date_jira_to_datetime(jira_date):
  regex = r"(\d{4})-(\d{2})-(\d{2})"
  match = re.match(regex, jira_date)

  if match:
      year, month, day = match.groups()
      datetime_object = datetime(year=int(year), month=int(month), day=int(day))
      return datetime_object
  else:
      return None

def convert_issues_to_dataframe(all_real_issues):
  l_issue_key_aux, l_issue_type_aux, l_issue_summary_aux, l_issue_description_aux, l_issue_status_aux, l_issue_priority_aux, l_issue_comments_aux = [], [], [], [], [], [], []
  l_issue_created_date, l_issue_resolved_date = [], []

  for issue in all_real_issues.get_issues():
    l_issue_key_aux.append(issue.key)
    l_issue_type_aux.append(issue.issue_type)
    l_issue_summary_aux.append(issue.summary)
    l_issue_description_aux.append(issue.description)
    l_issue_status_aux.append(issue.status)
    l_issue_priority_aux.append(issue.priority)
    texto_aux = ""
    for item in issue.get_comments():
      texto_aux = texto_aux + str(item) + "\n"
    l_issue_comments_aux.append(texto_aux)
    created_date_temp = convert_date_jira_to_datetime(issue.created_date)
    l_issue_created_date.append(created_date_temp)
    resolved_date_temp = convert_date_jira_to_datetime(issue.resolved_date)
    l_issue_resolved_date.append(resolved_date_temp)

  dict_all_reall_issues_in_commits_detailed = {
  'issue_key': l_issue_key_aux,
  'issue_type':l_issue_type_aux,
  'status':l_issue_status_aux,
  'priority':l_issue_priority_aux,
  'summary':l_issue_summary_aux,
  'description':l_issue_description_aux,
  'comments':l_issue_comments_aux,
  'created_date': l_issue_created_date,
  'resolved_date': l_issue_resolved_date
  }

  df_all_reall_issues_in_commits_detailed = pd.DataFrame(dict_all_reall_issues_in_commits_detailed)
  return df_all_reall_issues_in_commits_detailed

In [ ]:
def convert_commits_to_dataframe(dict_of_commits):
  '''
  v[0] = commit.msg,
  v[1] = data_commit,
  v[2] = commit.lines,
  v[3] = commit.files,
  v[4] = list_of_critical_files_modified,
  v[5] = list_of_modified_files,
  v[6] = list_dict_of_diff_files,
  v[7] = list_dict_of_diff_modified_files
  '''
  l_commit_hash, l_commit_msg, l_commit_data, l_commit_lines, l_commit_files, l_commit_critical_files, l_commit_modified_fies, l_commit_diff_files, l_commit_diff_modified_files = [], [], [], [], [], [], [], [], []
  for k, v in dict_of_commits.items():
    l_commit_hash.append(k)
    l_commit_msg.append(v[0])
    l_commit_data.append(v[1])
    l_commit_lines.append(v[2])
    l_commit_files.append(v[3])
    l_commit_critical_files.append(v[4])
    l_commit_modified_fies.append(v[5])
    l_commit_diff_files.append(v[6])
    l_commit_diff_modified_files.append(v[7])

  dict_of_commits_aux = {
      'hash': l_commit_hash,
      'msg': l_commit_msg,
      'date': l_commit_data,
      'lines': l_commit_lines,
      'files': l_commit_files,
      'critical_files': l_commit_critical_files,
      'modified_files': l_commit_modified_fies,
      'diff_files': l_commit_diff_files,
      'diff_files_modified_files': l_commit_diff_modified_files
  }

  df_commits = pd.DataFrame(dict_of_commits_aux)
  return df_commits

In [ ]:
def find_issues_id_by_project(input_string: str, project: str) -> list[str]:
    """Finds all Cassandra issue ID patterns in the input string.
    Args:
        input_string: The input string.
        project: The pattern related to project name, for example: CASSANDRA project name
    Returns:
        A list of project issue IDs, if found; otherwise, an empty list.
    """
    # Try to find all Cassandra issue ID patterns in the input string
    matches = re.findall(r"({0}-\d+)".format(project), input_string)
    # Return an empty list if no matches are found
    if not matches:
        return []
    # Convert the list of matches to a set to remove duplicates
    set_matches = set(matches)
    # Convert the set of matches back to a list
    list_unique_matches = list(set_matches)

    # Return the list of matched Cassandra issue IDs
    return list_unique_matches

def get_commits_with_critical_files_and_issues_in_this_commits(df_commits_with_critical_files):
  dict_issues_in_commits = {}
  for index in df_commits_with_critical_files.index:
    l_issues_in_commit = find_issues_id_by_project(input_string=df_commits_with_critical_files.msg[index], project=ISSUE_TRACKER_PROJECT)
    if len(l_issues_in_commit) > 0:
      commit_hash = df_commits_with_critical_files.hash[index]
      dict_issues_in_commits[commit_hash] = l_issues_in_commit

  list_issue_commits, list_issue_issues = [], []
  for k, v in dict_issues_in_commits.items():
    list_issue_commits.append(k)
    for issue in v:
      if issue not in list_issue_issues:
        list_issue_issues.append(issue)

  df_aux  = df_all_reall_issues_in_commits_detailed.copy()
  df_issues_in_commits_with_critical_classes = df_aux[df_aux['issue_key'].isin(list_issue_issues)]

  return dict_issues_in_commits, df_issues_in_commits_with_critical_classes

def calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size):
    # Calculate the Z-score for the given confidence level
    z_score = stats.norm.ppf(1 - (1 - confidence_level) / 2)

    # Calculate the sample size formula
    sample_size = ((z_score**2) * population_proportion * (1 - population_proportion)) / (margin_of_error**2)

    # Adjust for finite population
    if population_size:
        sample_size = sample_size / (1 + ((sample_size - 1) / population_size))

    return int(sample_size)

In [ ]:
def get_max_n_chars(text, max_n):
  text_length = len(text)
  if text_length <= max_n:
    return text
  else:
    return text[:max_n]

def create_new_file(filename, dir_name, issue_type, summary, description, status, comments):
  try:
    filename = dir_name + '/' + filename

    if issue_type is None:
      issue_type = ''
    if summary is None:
      summary = ''
    if description is None:
      description = ''
    if status is None:
      status = ''
    if comments is None:
      comments = ''

    with open(filename, mode='w') as f_issue:
      f_issue.write(f'issue_type: {issue_type} \n')
      f_issue.write(f'summary: {summary} \n')
      f_issue.write(f'description: {get_max_n_chars(text=description, max_n=1000)} \n')
      f_issue.write(f'status: {status} \n')
      f_issue.write(f'comments: {get_max_n_chars(text=comments, max_n=4000)} \n')
    print(f'File {filename} created with success!')

  except Exception as ex:
    print(f'Erro ao criar arquivo: {str(ex)}')


In [ ]:
# Seleciona randomicamente os issues para inspeção
def select_issues_to_inspection(sample_size, df_issues_in_commits_with_critical_classes, my_date='02/11/2023'):
  lista_issues_inspecao = []
  dict_issues_para_inspecao = {}
  list_issue_key = df_issues_in_commits_with_critical_classes.issue_key.to_list()
  list_issue_key = list(set(list_issue_key))
  sample_issues = random.choices(list_issue_key, k=sample_size)
  dict_issues_para_inspecao['02/11/2023'] = sample_issues
  print(f'{len(sample_issues)} para inspeção manual')

  date_file_name = my_date.split('/')
  date_file_name = date_file_name[0] + date_file_name[1] + date_file_name[2]
  file_name = 'issues_inspecao_' + date_file_name + '.txt'
  with open(file_name, mode='w') as f_temp:
    for v in dict_issues_para_inspecao[my_date]:
      elemento = v + ','
      f_temp.write(elemento)
  print(f'Relação de Issues salvos em {my_date} para inspeção.')
  return sample_issues

# Gera os arquivos .txt de cada issue selecionado para inspeção
def generate_files_issues_to_inspection(sample_issues, df_issues_in_commits_with_critical_classes):
  contador = 0
  my_dir_name = 'my_issues'
  if not os.path.exists(my_dir_name):
    os.makedirs(my_dir_name)

  total_of_issues = df_issues_in_commits_with_critical_classes.shape[0]
  for index in tqdm.tqdm(df_issues_in_commits_with_critical_classes.index, total=total_of_issues, desc='Analyzing issues'):
    for issue in sample_issues:
      if df_issues_in_commits_with_critical_classes.issue_key[index] == issue:
        create_new_file(filename=df_issues_in_commits_with_critical_classes.issue_key[index], dir_name=my_dir_name, issue_type=df_issues_in_commits_with_critical_classes.issue_type[index], summary=df_issues_in_commits_with_critical_classes.summary[index], description=df_issues_in_commits_with_critical_classes.description[index], status=df_issues_in_commits_with_critical_classes.status[index], comments=df_issues_in_commits_with_critical_classes.comments[index])
        contador += 1
  print(f'Foram criados {contador} arquivos para inspeção')

# 1. Extrai commits de um repositório de código

In [ ]:
start_date = datetime(2009, 1, 1, 0, 0, 0)
end_date = datetime(2023, 10, 4, 0, 0, 0)
extract_all_commits = get_all_commits_by_range(initial_date=start_date, final_date=end_date, repository_name=my_repository)

all_commits = extract_all_commits[0]
total_all_commits = extract_all_commits[1]

print(f'Total de commits extraídos: {total_all_commits}')

Wait...


Progress commit anlysis: 100%|██████████| 7941/7941 [01:40<00:00, 79.11it/s]

Total de commits extraídos: 7941


## Filtra os commits com classes críticas

In [ ]:
print(f'Analisa a faixa de commits entre: initial_date={str(start_date)}, final_date={str(end_date)}')
print('Registra apenas os commits que contem pelo menos um arquivo crítico')
filter_commits_with_critical_files = get_commits_by_range_and_critical_files(initial_date=start_date, final_date=end_date, critical_files=lista_arquivos_criticos, repository_name=my_repository)
commits_with_critical_files = filter_commits_with_critical_files[0]
total_of_commits_with_critical_files = filter_commits_with_critical_files[1]

print(f'Total de commits com classes críticas: {total_of_commits_with_critical_files}')

Analisa a faixa de commits entre: initial_date=2009-01-01 00:00:00, final_date=2023-10-04 00:00:00
Registra apenas os commits que contem pelo menos um arquivo crítico
Wait...


Progress critical files commit anlysis: 100%|██████████| 7941/7941 [00:54<00:00, 144.52it/s]

Total de commits com classes críticas: 7941


### Converte os commits com classes críticas para dataframe

In [ ]:
df_commits_with_critical_files = convert_commits_to_dataframe(dict_of_commits=commits_with_critical_files)
df_commits_with_critical_files

,hash,msg,date,lines,files,critical_files,modified_files,diff_files,diff_files_modified_files
0,07b6a38336a62913d42817a89cdc821a89419b7f,AMQ-2053 introduce methods to remove (closed) ...,7/1/2009,164,12,[Queue.java],"[AbstractRegion.java, DestinationFactory.java,...","[{'Queue.java': '@@ -850,9 +850,9 @@ public cl...","[@@ -203,7 +203,7 @@ public abstract class Abs..."
1,807f64591dcd4e0e496dc5795405812129d0d062,Recovery dispatch refactoring as the part of t...,22/1/2009,54,2,"[Queue.java, QueueDispatchSelector.java]","[Queue.java, QueueDispatchSelector.java]","[{'Queue.java': '@@ -233,16 +233,18 @@ public ...","[@@ -233,16 +233,18 @@ public class Queue exte..."
2,a321204079f435deaf8d082fb3de95c5c792c7f2,fix for http://issues.apache.org/activemq/brow...,23/1/2009,317,3,[Queue.java],"[Queue.java, PolicyEntry.java, MessageGroupDel...","[{'Queue.java': '@@ -27,12 +27,16 @@ import ja...","[@@ -27,12 +27,16 @@ import java.util.LinkedLi..."
3,977393d9e74364c21260f00d2fe0e314c77f9afa,fix for http://issues.apache.org/activemq/brow...,23/1/2009,170,4,[QueueDispatchSelector.java],"[QueueDispatchSelector.java, JmsTestSupport.ja...","[{'QueueDispatchSelector.java': '@@ -70,7 +70,...","[@@ -70,7 +70,7 @@ public class QueueDispatchS..."
4,39dd6b914a6954bf8f7114e97831482fd616e8bb,fix for https://issues.apache.org/activemq/bro...,27/1/2009,238,4,[DemandForwardingBridgeSupport.java],"[DemandForwardingBridgeSupport.java, TestBroke...",[{'DemandForwardingBridgeSupport.java': '@@ -4...,"[@@ -441,6 +441,9 @@ public abstract class Dem..."
...,...,...,...,...,...,...,...,...,...
716,9a5b61f6a28184cfe832871302ece16069ebb71d,[AMQ-9257] Disabled expire message checking wh...,19/5/2023,75,3,"[Queue.java, QueueDispatchSelector.java]","[Queue.java, QueueDispatchSelector.java, Expir...","[{'Queue.java': '@@ -957,6 +957,11 @@ public c...","[@@ -957,6 +957,11 @@ public class Queue exten..."
717,901956d4ddb6a0ea9fe5fedf39732117ab68f087,AMQ-9262 - Fix network subscriptions for compo...,7/6/2023,619,5,[DemandForwardingBridgeSupport.java],"[DemandForwardingBridgeSupport.java, Composite...",[{'DemandForwardingBridgeSupport.java': '@@ -2...,"[@@ -24,6 +24,7 @@ import java.util.Collection..."
718,95e3c2a1aa0d4962ad6a22a4422a9314875f824b,[AMQ-9233] Prevent NPE in SubQueueSelectorCach...,22/6/2023,2,1,[SubQueueSelectorCacheBroker.java],[SubQueueSelectorCacheBroker.java],[{'SubQueueSelectorCacheBroker.java': '@@ -180...,"[@@ -180,7 +180,7 @@ public class SubQueueSele..."
719,3ba93ed9579bbe90967312fba78726d96239110a,Remove unnecessary boxing / unboxing,23/7/2023,1307,111,"[DurableTopicSubscription.java, Queue.java]","[InboundTransformer.java, AmqpConnection.java,...","[{'DurableTopicSubscription.java': '@@ -227,9 ...","[@@ -90,7 +90,7 @@ public abstract class Inbou..."


In [ ]:
df_commits_with_critical_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721 entries, 0 to 720
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   hash                       721 non-null    object
 1   msg                        721 non-null    object
 2   date                       721 non-null    object
 3   lines                      721 non-null    int64 
 4   files                      721 non-null    int64 
 5   critical_files             721 non-null    object
 6   modified_files             721 non-null    object
 7   diff_files                 721 non-null    object
 8   diff_files_modified_files  721 non-null    object
dtypes: int64(2), object(7)
memory usage: 50.8+ KB


In [ ]:
df_commits_with_critical_files.to_excel('activemq_commits_with_critical_classes.xlsx', index=False)

# 2. Extrai issues de um issue tracker

## Configura ambiente para acessar Issue Tracker

In [ ]:
# Initialize the Jira connection
print('Initialize the Jira connection')
jira = JIRA(JIRA_SERVER, basic_auth=(username, password))

# Create a JiraUtils instance
print('Create a JiraUtils instance')
jira_utils = JiraUtils(ISSUE_TRACKER_PROJECT, jira)

date1 = (2009, 1, 1)
date2 = (2023, 10, 4)
distance = 120
print('Define date intervals')
print(f'From: {str(date1)} to: {date2}, by: {distance} days of distance.')

Initialize the Jira connection
Create a JiraUtils instance
Define date intervals
From: (2009, 1, 1) to: (2023, 10, 4), by: 120 days of distance.


## Recupera os issues do período dado

In [ ]:
# Fetch issues using date intervals
print('Fetch issues using date intervals')
block_of_issues = jira_utils.get_list_of_block_issues_by_dates(date1, date2, distance)

Fetch issues using date intervals
Aguarde...


Progress Message Analysis:   2%|▏         | 1/45 [00:02<01:51,  2.53s/it]

Range: project=AMQ and created>="2009/01/01" and created<="2009/04/30", qtd issues: 184


Progress Message Analysis:   4%|▍         | 2/45 [00:04<01:27,  2.04s/it]

Range: project=AMQ and created>="2009/05/01" and created<="2009/08/28", qtd issues: 129


...

Range: project=AMQ and created>="2022/10/20" and created<="2023/02/16", qtd issues: 63


Progress Message Analysis:  98%|█████████▊| 44/45 [01:22<00:01,  1.04s/it]

Range: project=AMQ and created>="2023/02/17" and created<="2023/06/16", qtd issues: 56


Progress Message Analysis: 100%|██████████| 45/45 [01:23<00:00,  1.84s/it]

Range: project=AMQ and created>="2023/06/17" and created<="2023/10/14", qtd issues: 47
2023-11-04 16:26:50.303982
Tempo da consulta: 0:01:23.013814


In [ ]:
# Concatenate the block of issues into a single list
print('Concatenate the block of issues into a single list')
all_issues = jira_utils.concatenate_block_of_issues(block_of_issues)

Concatenate the block of issues into a single list
Total de issues recuperados: 5955


In [ ]:
all_real_issues = analyze_jira_all_issues(ISSUE_TRACKER_PROJECT, all_issues)

Progress jira all issues analysis: 100%|██████████| 5955/5955 [00:00<00:00, 116114.14it/s]


### Converte os issues recuperados para um dataframe

In [ ]:
df_all_reall_issues_in_commits_detailed = convert_issues_to_dataframe(all_real_issues)
df_all_reall_issues_in_commits_detailed

,issue_key,issue_type,status,priority,summary,description,comments,created_date,resolved_date
0,AMQ-2300,New Feature,Resolved,Major,Add a new property for the camel component Act...,See discussion with James Strachan here : http...,Moved this ticket to AMQ as the camel-activemq...,2009-03-17,2013-11-02
1,AMQ-2234,Bug,Closed,Major,ActiveMQ can not be stopped sometimes via acti...,I use activemq-admin.bat for stopping activem...,Conf files and logs\nRecommend trying newer br...,2009-04-29,2012-12-13
2,AMQ-2233,Bug,Resolved,Major,After rollback received messages not re-presented,After rollback received messages not re-presen...,can you attach your test case (see the attach ...,2009-04-28,2009-07-08
3,AMQ-2232,Bug,Resolved,Minor,JMX Credentials Not Respected When Trying To M...,"On a secured broker, with secured JMX, and sec...",Fixed in svn revision 781022 by adding checks ...,2009-04-27,2009-06-02
4,AMQ-2231,Improvement,Resolved,Trivial,"Adjust POM to remove ""platform dependent"" warn...",Attached is a patch that specifies the source ...,Fixed by SVN revision 905967\n,2009-04-27,2010-02-03
...,...,...,...,...,...,...,...,...,...
5950,AMQ-9288,Task,Open,Major,TransportConnection Async error message needs ...,This:\r\n{noformat}\r\nif (SERVICELOG.isDebugE...,Commit b751428ebbf1d202bfa007e1836fc2368a791ef...,2023-06-27,NaT
5951,AMQ-9287,Bug,Resolved,Major,activemq 5.18.1 with jdk 17,While attempting to use ActiveMQ 5.18.1 with J...,ActiveMQ 5.18.x is JDK11 only.\r\n\r\nActiveMQ...,2023-06-26,2023-10-24
5952,AMQ-9286,Dependency upgrade,Resolved,Major,Upgrade to Apache POM 30,,Commit d4011bc16431ef890a475ee876950c97d7cadb8...,2023-06-25,2023-06-25
5953,AMQ-9285,Bug,Resolved,Trivial,User is informed to inspect missing log4j.prop...,The following is printed out when ActiveMQ is ...,Commit 58246c8a16aa5c55ee3da70113c21f9c98285d8...,2023-06-25,2023-06-26


In [ ]:
colunas = ['issue_key',	'issue_type', 'status', 'priority', 'summary']
df_all_reall_issues_in_commits_detailed[colunas].to_excel('activemq_all_issues_in_commits.xlsx', index=False)

In [ ]:
df_all_reall_issues_in_commits_detailed['time_resolution'] = df_all_reall_issues_in_commits_detailed['resolved_date'] - df_all_reall_issues_in_commits_detailed['created_date']

In [ ]:
print('Gera arquivo .csv activemq_all_issues_in_commits')
colunas = ['issue_key', 'issue_type', 'status', 'summary','created_date', 'resolved_date', 'time_resolution']
df_all_reall_issues_in_commits_detailed[colunas].to_csv('activemq_all_issues_in_commits.csv', index=False)

Gera arquivo .csv activemq_all_issues_in_commits


### Recupera Issues com referência nos commits com classes críticas

In [ ]:
dict_issues_in_commits, df_issues_in_commits_with_critical_classes = get_commits_with_critical_files_and_issues_in_this_commits(df_commits_with_critical_files)
df_issues_in_commits_with_critical_classes

,issue_key,issue_type,status,priority,summary,description,comments,created_date,resolved_date,time_resolution
36,AMQ-2199,Bug,Resolved,Major,NullPointerException during broker start on br...,I have a configuration where I first start app...,Came across this behaviour using the debugger ...,2009-04-08,2009-04-09,1 days
37,AMQ-2198,Improvement,Resolved,Major,Allow suppression of duplicate queue subscript...,in a cyclic network of brokers (where each bro...,resolved in r762925\n\nAddition of suppressDup...,2009-04-07,2009-04-07,0 days
52,AMQ-2183,Bug,Resolved,Major,Master/slave out of sync with multiple consume...,1. producer of VirtualTopic.testTopic publish ...,Attached patch fix the issue. I create this ne...,2009-03-25,2010-04-27,398 days
55,AMQ-2180,Improvement,Resolved,Major,infinite networkTTL,"Hello,\n\nI would like to set a networkTTL wit...",Set the NetworkTTL to a value as large as the ...,2009-03-24,2010-06-16,449 days
86,AMQ-2149,Bug,Resolved,Major,Shared Filesystem Master Slave: missing messages,I'm finding occasionally messages are not deli...,I tried several more tests using the script an...,2009-03-06,2009-05-23,78 days
...,...,...,...,...,...,...,...,...,...,...
5846,AMQ-9153,Bug,Resolved,Major,Fix Slow Consumer Advisory for Queue subscript...,The {{AdvisoryTests}} class was not running du...,It looks like this has been broken for 10 year...,2022-11-01,2022-11-01,0 days
5872,AMQ-9262,Bug,Resolved,Major,Composite consumers do not work properly with ...,I found an interesting edge case where consume...,Commit 901956d4ddb6a0ea9fe5fedf39732117ab68f08...,2023-06-06,2023-06-07,1 days
5877,AMQ-9257,Improvement,Resolved,Minor,Disabled expire message checking when pauseDis...,"If pauseDispatch is set, the expiry checking s...",What is the background here? This ticket needs...,2023-05-10,2023-06-19,40 days
5895,AMQ-9239,New Feature,Closed,Major,Jakarta JMS package namespace for broker,,Commit 6b277a49037fd886e4b1c969c8cff97add335dc...,2023-03-30,2023-08-30,153 days


In [ ]:
df_issues_in_commits_with_critical_classes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 36 to 5901
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   issue_key        480 non-null    object         
 1   issue_type       480 non-null    object         
 2   status           480 non-null    object         
 3   priority         480 non-null    object         
 4   summary          480 non-null    object         
 5   description      480 non-null    object         
 6   comments         480 non-null    object         
 7   created_date     480 non-null    datetime64[ns] 
 8   resolved_date    476 non-null    datetime64[ns] 
 9   time_resolution  476 non-null    timedelta64[ns]
dtypes: datetime64[ns](2), object(7), timedelta64[ns](1)
memory usage: 41.2+ KB


In [ ]:
df_issues_in_commits_with_critical_classes['time_resolution'] = df_issues_in_commits_with_critical_classes['resolved_date'] - df_issues_in_commits_with_critical_classes['created_date']

In [ ]:
print('Gera arquivo .csv activemq_issues_in_commits_arquivos_criticos')
df_issues_in_commits_with_critical_classes[colunas].to_csv('activemq_issues_in_commits_with_critical_classes.csv', index=False)

Gera arquivo .csv activemq_issues_in_commits_arquivos_criticos


In [ ]:
df_issues_in_commits_with_critical_classes[colunas].to_excel('activemq_issues_in_commits_with_critical_classes.xlsx', index=False)

# Seleciona amostra dos Issues

In [ ]:
confidence_level = 0.95
margin_of_error = 0.05
population_proportion = 0.8
population_size = len(df_issues_in_commits_with_critical_classes)

sample_size = calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size)
print(f'População de issues: {population_size}')
print(f"Sample size para inspeção: {sample_size}")

População de issues: 480
Sample size para inspeção: 162


In [ ]:
sample_issues = select_issues_to_inspection(sample_size, df_issues_in_commits_with_critical_classes, my_date='02/11/2023')

162 para inspeção manual
Relação de Issues salvos em 02/11/2023 para inspeção.


In [ ]:
generate_files_issues_to_inspection(sample_issues, df_issues_in_commits_with_critical_classes)

Analyzing issues:   0%|          | 0/480 [00:00<?, ?it/s]

File my_issues/AMQ-2183 created with success!
File my_issues/AMQ-2091 created with success!
File my_issues/AMQ-2343 created with success!
File my_issues/AMQ-2327 created with success!
File my_issues/AMQ-2322 created with success!
File my_issues/AMQ-2321 created with success!
File my_issues/AMQ-2314 created with success!
File my_issues/AMQ-2314 created with success!
File my_issues/AMQ-2283 created with success!
File my_issues/AMQ-2262 created with success!
File my_issues/AMQ-2245 created with success!
File my_issues/AMQ-2484 created with success!
File my_issues/AMQ-2484 created with success!


Analyzing issues:  11%|█         | 52/480 [00:00<00:00, 510.10it/s]

File my_issues/AMQ-2468 created with success!
File my_issues/AMQ-2435 created with success!
File my_issues/AMQ-2403 created with success!
File my_issues/AMQ-2400 created with success!
File my_issues/AMQ-2704 created with success!
File my_issues/AMQ-2632 created with success!
File my_issues/AMQ-2610 created with success!
File my_issues/AMQ-2574 created with success!
File my_issues/AMQ-2572 created with success!
File my_issues/AMQ-2571 created with success!
File my_issues/AMQ-2870 created with success!
File my_issues/AMQ-2719 created with success!
File my_issues/AMQ-3020 created with success!
File my_issues/AMQ-3004 created with success!


Analyzing issues:  22%|██▏       | 106/480 [00:00<00:00, 523.98it/s]

...

File my_issues/AMQ-6441 created with success!
File my_issues/AMQ-6406 created with success!
File my_issues/AMQ-6381 created with success!
File my_issues/AMQ-6577 created with success!
File my_issues/AMQ-6707 created with success!
File my_issues/AMQ-6703 created with success!
File my_issues/AMQ-6703 created with success!
File my_issues/AMQ-6687 created with success!
File my_issues/AMQ-6667 created with success!
File my_issues/AMQ-6665 created with success!
File my_issues/AMQ-6849 created with success!
File my_issues/AMQ-6847 created with success!
File my_issues/AMQ-6788 created with success!
File my_issues/AMQ-6788 created with success!
File my_issues/AMQ-6880 created with success!
File my_issues/AMQ-6947 created with success!
File my_issues/AMQ-7035 created with success!
File my_issues/AMQ-7142 created with success!


Analyzing issues:  94%|█████████▍| 450/480 [00:00<00:00, 541.43it/s]

File my_issues/AMQ-7228 created with success!
File my_issues/AMQ-7193 created with success!
File my_issues/AMQ-7185 created with success!
File my_issues/AMQ-7302 created with success!
File my_issues/AMQ-7301 created with success!
File my_issues/AMQ-7493 created with success!


Analyzing issues: 100%|██████████| 480/480 [00:00<00:00, 547.15it/s]

File my_issues/AMQ-7438 created with success!
File my_issues/AMQ-8117 created with success!
File my_issues/AMQ-8316 created with success!
File my_issues/AMQ-8413 created with success!
File my_issues/AMQ-8976 created with success!
File my_issues/AMQ-9157 created with success!
Foram criados 162 arquivos para inspeção


In [ ]:
!zip -r my_issues.zip my_issues

  adding: my_issues/ (stored 0%)
  adding: my_issues/AMQ-6171 (deflated 39%)
  adding: my_issues/AMQ-4924 (deflated 58%)
  adding: my_issues/AMQ-5659 (deflated 62%)
  adding: my_issues/AMQ-5813 (deflated 53%)
  adding: my_issues/AMQ-2183 (deflated 52%)
  adding: my_issues/AMQ-6406 (deflated 53%)
  adding: my_issues/AMQ-5417 (deflated 44%)
  adding: my_issues/AMQ-6331 (deflated 52%)
  adding: my_issues/AMQ-6247 (deflated 45%)
  adding: my_issues/AMQ-5070 (deflated 38%)
  adding: my_issues/AMQ-3020 (deflated 42%)
  adding: my_issues/AMQ-3733 (deflated 47%)
  adding: my_issues/AMQ-6256 (deflated 64%)
  adding: my_issues/AMQ-8976 (deflated 29%)
  adding: my_issues/AMQ-7438 (deflated 63%)
  adding: my_issues/AMQ-4638 (deflated 26%)
  adding: my_issues/AMQ-3362 (deflated 46%)
  adding: my_issues/AMQ-6500 (deflated 61%)
  adding: my_issues/AMQ-3609 (deflated 39%)
  adding: my_issues/AMQ-2314 (deflated 49%)
  adding: my_issues/AMQ-4656 (deflated 71%)
  adding: my_issues/AMQ-2610 (deflated 48%)